In [34]:
from dotenv import load_dotenv
load_dotenv()

True

In [35]:
#Connecting to SQL Database 'Chinook.db' 

from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///chinook.db")

print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [37]:
#Setting up Chains 
# 1. Convert Question into SQL query
# 2. Execute SQL query
# 3. Ue result, generate answer 

#Chain #1 -> Convert Question into SQL query

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o-mini")

from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many artist are there"})
response


'SQLQuery: SELECT COUNT(*) AS "ArtistCount" FROM "Artist";'

In [45]:
#Chain #2 -> Execute SQL query (No human approval for querying -> to add for the future!)

from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm,db)

chain = write_query | execute_query
chain.invoke({"question": "How many artist are there"})

# I MUST FIGURE OUT THIS ERROR!!!!!!!!!

'Error: (sqlite3.OperationalError) near "SQLQuery": syntax error\n[SQL: SQLQuery: SELECT COUNT(*) AS "ArtistCount" FROM "Artist";]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

In [49]:
#Chain #3 -> Answer original prompt

from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question and fix the query to the correct SQL query should an error arise.
    
    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(result=itemgetter("query") | execute_query) | answer_prompt | llm | StrOutputParser()
)

chain.invoke({"question": "How many employees are there"})

'The user asked how many employees there are, and the SQL query has a syntax error. \n\nThe error arises from the incorrect use of backticks and the inclusion of "SQLQuery:" in the SQL statement. The correct SQL query should not include any extraneous text or formatting.\n\nHere’s the corrected SQL query:\n\n```sql\nSELECT COUNT("EmployeeId") AS "NumberOfEmployees" FROM "Employee"\n```\n\nWith this corrected query, it will properly count the number of employees in the "Employee" table. \n\n### Answer:\nThere are [number] employees. (Replace [number] with the result of the corrected SQL query.)'